In [ ]:
!pip install rectools
!pip install lightfm
!pip install optuna
!pip install annoy

In [ ]:
import os

In [ ]:
import requests
import pandas as pd 
import numpy as np

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import LightFMWrapperModel

import matplotlib.pyplot as plt

import typing as tp
from tqdm import tqdm

from lightfm import LightFM

import optuna

from annoy import AnnoyIndex

In [ ]:
# download dataset by chunks
url = "https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip"

req = requests.get(url, stream=True)

with open('kion_train.zip', "wb") as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)


kion dataset download: 100%|██████████| 78.8M/78.8M [2:29:22<00:00, 8.79kiB/s]
kion dataset download: 100%|█████████▉| 78.6M/78.8M [00:05<00:00, 20.2MiB/s]

In [ ]:
!unzip kion_train.zip

In [ ]:
interactions = pd.read_csv('kion_train/interactions.csv')
users = pd.read_csv('kion_train/users.csv')
items = pd.read_csv('kion_train/items.csv')

## Normalization

Нормализуем DataFrame с пользователями для того, чтобы затем применить ANN.

In [ ]:
users = users.dropna()
users['sex'] = users['sex'].map({'Ж': 1, 'М': 0})
users['age'] = users['age'].map({
    'age_18_24': 0,
    'age_25_34': 1,
    'age_35_44': 2,
    'age_45_54': 3,
    'age_55_64': 4,
    'age_65_inf': 5
})

users['income'] = users['income'].map({
    'income_0_20': 0,
    'income_20_40': 1,
    'income_40_60': 2,
    'income_60_90': 3,
    'income_90_150': 4,
    'income_150_inf': 5,    
})

In [ ]:
Columns.Datetime = 'last_watch_dt'

In [ ]:
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)

kion dataset download: 100%|██████████| 78.8M/78.8M [00:24<00:00, 20.2MiB/s]

## Train/test split

In [ ]:
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')

In [ ]:
max_date = interactions[Columns.Datetime].max()

In [ ]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

In [ ]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (4985269, 6)
test: (490982, 6)


In [ ]:
train.drop(train.query("total_dur < 300").index, inplace=True)

In [ ]:
# отфильтруем холодных пользователей из теста
cold_users = set(test[Columns.User]) - set(train[Columns.User])

In [ ]:
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

# MODELS

In [ ]:
K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 16
N_EPOCHS = 10

N_TRIALS = 2 # number of iterations for optuna to tune hyperparameters 

In [ ]:
dataset = Dataset.construct(
    interactions_df=train
)

Было взято 3 модели, созданные при помощи LightFM и подобраны гиперпараметры learning rate, количество компонентов и функция потерь. Каждая модель оптимизировалась под различные метрики: "MAP@10", "Precision@10", "Recall@10" соответственно.

Ниже представлена модель, для которой гиперпараметры подбирались для максимизации метрики Mean Average Precision (при k=10):

In [ ]:
metric_results = []

In [ ]:
def objective_MAP10(trial, dataset):
  param_grid = {
      "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.4),
      "no_components": trial.suggest_int("no_components", 4, 32, step=4),
      "loss": trial.suggest_categorical('loss', ['logistic', 'bpr', 'warp'])
  }

  metrics_name = {
    'Precision': Precision,
    'Recall': Recall,
    'MAP': MAP,
  }

  LightFM_model = LightFMWrapperModel(
      LightFM(
          **param_grid,
          random_state=RANDOM_STATE,
      ),
        epochs=N_EPOCHS,
        num_threads=NUM_THREADS,
  )
  
  metrics = {}

  for metric_name, metric in metrics_name.items():
      metrics[f'{metric_name}@{K_RECOS}'] = metric(k=K_RECOS)

  LightFM_model.fit(dataset)

  recos = LightFM_model.recommend(
        users=test[Columns.User].unique(),
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
  )
  metric_values = calc_metrics(metrics, recos, test, train)
  metric_results.append(metric_values)
  return metric_values['MAP@10']
  

In [ ]:
study_map = optuna.create_study(direction = "maximize", study_name = "LightFM_MAP")  # Create a new study.
func = lambda trial: objective_MAP10(trial, dataset)
study_map.optimize(func, n_trials=N_TRIALS, show_progress_bar=True)

[I 2022-12-12 23:35:04,777] A new study created in memory with name: LightFM_MAP
/usr/local/lib/python3.8/dist-packages/optuna/progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/2 [00:00<?, ?it/s]

kion dataset download: 100%|██████████| 78.8M/78.8M [04:23<00:00, 20.2MiB/s]

[I 2022-12-12 23:38:50,630] Trial 0 finished with value: 0.006992761942281843 and parameters: {'learning_rate': 0.22414127684327484, 'no_components': 32, 'loss': 'warp'}. Best is trial 0 with value: 0.006992761942281843.


kion dataset download: 100%|██████████| 78.8M/78.8M [07:30<00:00, 20.2MiB/s]

[I 2022-12-12 23:41:57,871] Trial 1 finished with value: 0.01639973837297493 and parameters: {'learning_rate': 0.26180767463016885, 'no_components': 28, 'loss': 'bpr'}. Best is trial 1 with value: 0.01639973837297493.


Ниже представлена модель, для которой гиперпараметры подбирались для максимизации метрики Precision (при k=10):

In [ ]:
metric_results_pr = []

In [ ]:
def objective_Precision10(trial, dataset):
  param_grid = {
      "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.4),
      "no_components": trial.suggest_int("no_components", 4, 32, step=4),
      "loss": trial.suggest_categorical('loss', ['logistic', 'bpr', 'warp'])
  }

  metrics_name = {
    'Precision': Precision,
    'Recall': Recall,
    'MAP': MAP,
  }

  LightFM_model = LightFMWrapperModel(
      LightFM(
          **param_grid,
          random_state=RANDOM_STATE,
      ),
        epochs=N_EPOCHS,
        num_threads=NUM_THREADS,
  )
  
  metrics = {}

  for metric_name, metric in metrics_name.items():
      metrics[f'{metric_name}@{K_RECOS}'] = metric(k=K_RECOS)

  LightFM_model.fit(dataset)

  recos = LightFM_model.recommend(
        users=test[Columns.User].unique(),
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
  )
  metric_values = calc_metrics(metrics, recos, test, train)
  metric_results_pr.append(metric_values)

  return metric_values['Precision@10']
  

In [ ]:
study_precision = optuna.create_study(direction = "maximize", study_name = "LightFM_Precision")  # Create a new study.
func = lambda trial: objective_Precision10(trial, dataset)
study_precision.optimize(func, n_trials=N_TRIALS, show_progress_bar=True)

[I 2022-12-12 23:41:57,938] A new study created in memory with name: LightFM_Precision
/usr/local/lib/python3.8/dist-packages/optuna/progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/2 [00:00<?, ?it/s]

kion dataset download: 100%|██████████| 78.8M/78.8M [10:39<00:00, 20.2MiB/s]

[I 2022-12-12 23:45:06,221] Trial 0 finished with value: 0.03479026704550484 and parameters: {'learning_rate': 0.09090673242044323, 'no_components': 32, 'loss': 'warp'}. Best is trial 0 with value: 0.03479026704550484.


kion dataset download: 100%|██████████| 78.8M/78.8M [13:49<00:00, 20.2MiB/s]

[I 2022-12-12 23:48:16,861] Trial 1 finished with value: 0.0029625498995370747 and parameters: {'learning_rate': 0.2471240695154021, 'no_components': 24, 'loss': 'warp'}. Best is trial 0 with value: 0.03479026704550484.


Ниже представлена модель, для которой гиперпараметры подбирались для максимизации метрики Recall (при k=10):

In [ ]:
metric_results_rec = []

In [ ]:
def objective_Recall10(trial, dataset):
  param_grid = {
      "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.4),
      "no_components": trial.suggest_int("no_components", 4, 32, step = 4),
      "loss": trial.suggest_categorical('loss', ['logistic', 'bpr', 'warp'])
  }

  metrics_name = {
    'Precision': Precision,
    'Recall': Recall,
    'MAP': MAP,
  }

  LightFM_model = LightFMWrapperModel(
      LightFM(
          **param_grid,
          random_state=RANDOM_STATE,
      ),
        epochs=N_EPOCHS,
        num_threads=NUM_THREADS,
  )
  
  metrics = {}

  for metric_name, metric in metrics_name.items():
      metrics[f'{metric_name}@{K_RECOS}'] = metric(k=K_RECOS)

  LightFM_model.fit(dataset)

  recos = LightFM_model.recommend(
        users=test[Columns.User].unique(),
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
  )
  metric_values = calc_metrics(metrics, recos, test, train)
  metric_results_rec.append(metric_values)

  return metric_values['Recall@10']
  

In [ ]:
study_recall = optuna.create_study(direction = "maximize", study_name = "LightFM_Recall")  # Create a new study.
func = lambda trial: objective_Recall10(trial, dataset)
study_recall.optimize(func, n_trials=N_TRIALS, show_progress_bar=True)

[I 2022-12-12 23:48:16,917] A new study created in memory with name: LightFM_Recall
/usr/local/lib/python3.8/dist-packages/optuna/progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/2 [00:00<?, ?it/s]

kion dataset download: 100%|██████████| 78.8M/78.8M [16:10<00:00, 20.2MiB/s]

[I 2022-12-12 23:50:38,094] Trial 0 finished with value: 2.60758446052798e-05 and parameters: {'learning_rate': 0.30337765773250575, 'no_components': 8, 'loss': 'warp'}. Best is trial 0 with value: 2.60758446052798e-05.


kion dataset download: 100%|██████████| 78.8M/78.8M [18:10<00:00, 20.2MiB/s]

[I 2022-12-12 23:52:37,670] Trial 1 finished with value: 0.15041396168251372 and parameters: {'learning_rate': 0.051011318510917905, 'no_components': 12, 'loss': 'logistic'}. Best is trial 1 with value: 0.15041396168251372.


Вывод лучших значений метрик и соответствующих параметров:

In [ ]:
print(f"\tBest value (MAP@10): {study_map.best_value:.5f}")
print(f"\tBest params (MAP@10):")

for key, value in study_map.best_params.items():
    print(f"\t\t{key}: {value}")

print(f"\tPrecision@10 in best result:")
best_result = list(filter(lambda best_result: best_result['MAP@10'] == study_map.best_value, metric_results))[0]
print(f"\t\t{best_result['Precision@10']}")
print(f"\tRecall@10 in best result:")
print(f"\t\t{best_result['Recall@10']}")

	Best value (MAP@10): 0.01640
	Best params (MAP@10):
		learning_rate: 0.26180767463016885
		no_components: 28
		loss: bpr
	Precision@10 in best result:
		0.009307204121192809
	Recall@10 in best result:
		0.03734535959192731


In [ ]:
print(f"\tBest value (Precision@10): {study_precision.best_value:.5f}")
print(f"\tBest params (Precision@10):")

for key, value in study_precision.best_params.items():
    print(f"\t\t{key}: {value}")

print(f"\tMAP@10 in best result:")
best_result = list(filter(lambda best_result: best_result['Precision@10'] == study_precision.best_value, metric_results_pr))[0]
print(f"\t\t{best_result['MAP@10']}")
print(f"\tRecall@10 in best result:")
print(f"\t\t{best_result['Recall@10']}")

	Best value (Precision@10): 0.03479
	Best params (Precision@10):
		learning_rate: 0.09090673242044323
		no_components: 32
		loss: warp
	MAP@10 in best result:
		0.07537917999018105
	Recall@10 in best result:
		0.15768086467626738


In [ ]:
print(f"\tBest value (Recall@10): {study_recall.best_value:.5f}")
print(f"\tBest params (Recall@10):")

for key, value in study_precision.best_params.items():
    print(f"\t\t{key}: {value}")

print(f"\tMAP@10 in best result:")
best_result = list(filter(lambda best_result: best_result['Recall@10'] == study_recall.best_value, metric_results_rec))[0]
print(f"\t\t{best_result['MAP@10']}")
print(f"\tPrecision@10 in best result:")
print(f"\t\t{best_result['Precision@10']}")

	Best value (Recall@10): 0.15041
	Best params (Recall@10):
		learning_rate: 0.09090673242044323
		no_components: 32
		loss: warp
	MAP@10 in best result:
		0.07467506502712265
	Precision@10 in best result:
		0.0318153252431911


На графике гиперпараметры отсортированы по значимости (для модели, у которой гиперпараметры подбирались по **MAP@10**):

In [ ]:
fig = optuna.visualization.plot_param_importances(study_map)
fig.show()

На графике гиперпараметры отсортированы по значимости (для модели, у которой гиперпараметры подбирались по **Precision@10**):

In [ ]:
fig = optuna.visualization.plot_param_importances(study_precision)
fig.show()

На графике гиперпараметры отсортированы по значимости (для модели, у которой гиперпараметры подбирались по **Recall@10**):

In [ ]:
fig = optuna.visualization.plot_param_importances(study_recall)
fig.show()

# ANN

In [ ]:
f = 4  # Length of item vector that will be indexed

ANN_tree = AnnoyIndex(f, 'angular')
for index, row in users.iterrows():
    i = row['user_id']
    v = row[['age', 'income', 'sex', 'kids_flg']]
    ANN_tree.add_item(i, v)

ANN_tree.build(32) # 32 trees

# "Аватары"

Фильмы, которые посмотрел **первый** пользователь (возраст - от 25 до 34 лет включительно, доход - от 40 до 60 тыс. рублей, мужчина, без детей):

1.   Хроники Нарнии: Лев, колдунья и волшебный шкаф
2.   Терминатор
3.   Звёздные войны: Эпизод 4 — Новая надежда


Фильмы, которые посмотрел **второй** пользователь (возраст - от 18 до 24 лет включительно, доход - от 20 до 40 тыс. рублей, мужчина, без детей):

1.   Каратэ-пацан
2.   Шаг вперёд
3.   Пираты карибского моря: На странных берегах


Фильмы, которые посмотрел **третий** пользователь (возраст - от 35 до 44 лет включительно, доход - свыше 150 тыс. рублей, мужчина, есть дети):

1.   Али
2.   Жизнь Пи
3.   Кон-Тики

За даты просмотров взято самая поздняя дата из датасета. Длина просмотра: 7000 секунд, 100% каждого фильма просмотрено (вес-3).



In [309]:
df = pd.DataFrame([
    [users['user_id'].max()+1,1,2,1,0],   
    [users['user_id'].max()+2,0,1,1,0],
    [users['user_id'].max()+3,2,5,1,1]],
    columns=['user_id','age','income', 'sex', 'kids_flg']
)


df2 = pd.DataFrame([
    [users['user_id'].max()+1,945,max_date,7000,100.0,3], 
    [users['user_id'].max()+1,6720,max_date,7000,100.0,3], 
    [users['user_id'].max()+1,8980,max_date,7000,100.0,3], 
    [users['user_id'].max()+2,11973,max_date,7000,100.0,3], 
    [users['user_id'].max()+2,13460,max_date,7000,100.0,3], 
    [users['user_id'].max()+2,12057,max_date,7000,100.0,3],
    [users['user_id'].max()+3,4405,max_date,7000,100.0,3],
    [users['user_id'].max()+3,14310,max_date,7000,100.0,3],
    [users['user_id'].max()+3,10351,max_date,7000,100.0,3]],
    columns=['user_id','item_id','last_watch_dt', 'total_dur', 'watched_pct', 'weight'])

users_added = pd.concat([df, users])
users_added_interactions = pd.concat([df2, interactions])

In [ ]:
users_added_interactions.head(9)

In [ ]:
users_added.head(3)

,user_id,age,income,sex,kids_flg
0,1097559,1,2,1,0
1,1097560,0,1,1,0
2,1097561,2,5,1,1


In [298]:
f = 4  # Length of item vector that will be indexed

ANN_tree = AnnoyIndex(f, 'angular')
for index, row in users_added.iterrows():
    i = row['user_id']
    v = row[['age', 'income', 'sex', 'kids_flg']]
    ANN_tree.add_item(i, v)

ANN_tree.build(32) # 32 trees

True

Берем фильмы, которые смотрели похожие пользователи, сортируем их по весу (длительности просмотра) и дате просмотра и рекомендуем.

Рекомендации ищутся в цикле, пока их не будет больше 8 функция не завершит работу.

In [300]:
def get_recomendations_ANN(user_id, n_nearest = 5):
  recs = pd.DataFrame()
  recs_titles = []
  i = 0
  while (len(recs) < 7):
    nearest_list = ANN_tree.get_nns_by_item(user_id, n_nearest)

    print("Рекомендуем пользователю:\n")
    print(users_added.loc[users_added['user_id']==user_id])
    print("\nКоторый смотрел следующие фильмы:\n")
    user_watched = users_added_interactions.loc[
        users_added_interactions['user_id'] == user_id
        ]['item_id']
    
    for item_id in user_watched:
      print(items.loc[items['item_id'] == item_id]['title'].to_list()[0])

    print('\n==============================\n')

    user_index = 0
    
    for id in nearest_list:
      # Чем ближе пользователь в ANN к исходному, тем рекомендации приоритетнее
      user_index+=1

      print("\nПользователь, похожий на данного:\n")
      print(users_added.loc[users_added['user_id'] == id])
      print("\nОн смотрел следующие фильмы:\n")
      df_nearest_user_interactions = users_added_interactions.loc[
          users_added_interactions['user_id'] == id
          ]['item_id']

      for item_id in df_nearest_user_interactions:
        current_user_rec = items.loc[items['item_id'] == item_id]['title'].to_list()[0]
        df = pd.DataFrame([
          [item_id,
           users_added_interactions.loc[
               (users_added_interactions['item_id']==item_id) & (users_added_interactions['user_id']==id)
               ]['last_watch_dt'].to_list()[0],
           users_added_interactions.loc[
               (users_added_interactions['item_id']==item_id) & (users_added_interactions['user_id']==id)
               ]['weight'].to_list()[0],
           user_index
           ]],
          columns=['item_id','last_watch_dt','weight','user_index']
        )
        recs = pd.concat([recs, df])
        print(current_user_rec)

    # убираем из рекомендаций фильмы, которые пользователи не смотрели достаточно времени
    recs = recs[recs.weight >= 2] 

    # сортируем по времени
    recs = recs.sort_values(['last_watch_dt', 'user_index'], ascending=[False, False])
    
    # повторяющиеся фильмы удаляем
    recs = recs.drop_duplicates(subset=['item_id']) 

    recs = recs.head(7)

    i+=1
    if i > 10:
      print('TIMEOUT: Не удалось найти полный список рекомендаций')

  recs = recs['item_id'].to_list()
  for item_id in recs:
    recs_titles.append(items.loc[items['item_id'] == item_id]['title'].to_list()[0])
  
  return recs_titles

Проверяем рекомендации для **первого** пользователя:

In [301]:
recs = get_recomendations_ANN(1097559)

Рекомендуем пользователю:

   user_id  age  income  sex  kids_flg
0  1097559    1       2    1         0

Который смотрел следующие фильмы:

Хроники Нарнии: Лев, колдунья и волшебный шкаф
Терминатор
Звёздные войны: Эпизод 4 — Новая надежда



Пользователь, похожий на данного:

        user_id  age  income  sex  kids_flg
285557     2815    1       2    1         0

Он смотрел следующие фильмы:

Путешествие времени
#Только серьёзные отношения
Дед, привет!
Стойкая броня
Kingsman: Золотое кольцо
Аладдин
Король лев (2019)

Пользователь, похожий на данного:

        user_id  age  income  sex  kids_flg
265760     3559    1       2    1         0

Он смотрел следующие фильмы:

Прабабушка легкого поведения
Секреты семейной жизни
Клиника счастья

Пользователь, похожий на данного:

        user_id  age  income  sex  kids_flg
716748     8114    1       2    1         0

Он смотрел следующие фильмы:

Секреты семейной жизни
Клиника счастья
Содержанки

Пользователь, похожий на данного:

        user_

In [302]:
recs

['Клиника счастья',
 'Содержанки',
 'Секреты семейной жизни',
 'Прабабушка легкого поведения',
 'Бывшая с того света',
 'Дуров',
 'Kingsman: Золотое кольцо']

Как видно, среди фильмов есть довольно хорошо подходящие: Дуров, Kingsman, Прабабушка легкого поведения. Нельзя назвать эти рекомендации идеальными, но они неплохие.

Проверяем рекоменндации для **второго** пользователя:

In [303]:
recs = get_recomendations_ANN(1097560)

Рекомендуем пользователю:

   user_id  age  income  sex  kids_flg
1  1097560    0       1    1         0

Который смотрел следующие фильмы:

Каратэ-пацан
Шаг вперёд
Пираты карибского моря: На странных берегах



Пользователь, похожий на данного:

        user_id  age  income  sex  kids_flg
235151      425    0       1    1         0

Он смотрел следующие фильмы:

Хрустальный
Круэлла

Пользователь, похожий на данного:

       user_id  age  income  sex  kids_flg
89443     5567    0       1    1         0

Он смотрел следующие фильмы:

Руслан и Людмила: перезагрузка
Перебежчик

Пользователь, похожий на данного:

        user_id  age  income  sex  kids_flg
274233     8625    0       1    1         0

Он смотрел следующие фильмы:

В постели с Викторией
Маша
Гнев человеческий
Непосредственно Каха
Собибор
Хрустальный
Приворот. Чёрное венчание

Пользователь, похожий на данного:

        user_id  age  income  sex  kids_flg
326233    12706    0       1    1         0

Он смотрел следующие фильмы

In [304]:
recs

['Красавица и чудовище',
 'Хрустальный',
 'Непосредственно Каха',
 'История игрушек 4',
 'Собибор',
 'Руслан и Людмила: перезагрузка',
 'Перебежчик']

Здесь были рекомендованы в основном российские фильмы, легкие, комедийные. В целом это похоже на то, что смотрел второй пользователь. Также, в топе рекомендаций мультфильм примерно тех же годов выпуска, что Пираты Карибского моря, это подходящая рекомендация.

In [305]:
recs = get_recomendations_ANN(1097561)

Рекомендуем пользователю:

   user_id  age  income  sex  kids_flg
2  1097561    2       5    1         1

Который смотрел следующие фильмы:

Али
Жизнь Пи
Кон-Тики



Пользователь, похожий на данного:

        user_id  age  income  sex  kids_flg
406055    79662    2       5    1         1

Он смотрел следующие фильмы:


Пользователь, похожий на данного:

        user_id  age  income  sex  kids_flg
171798   953802    2       5    1         1

Он смотрел следующие фильмы:

Шугалей 2
Гнев человеческий

Пользователь, похожий на данного:

        user_id  age  income  sex  kids_flg
794953    70984    2       5    0         1

Он смотрел следующие фильмы:

Миньоны
Гнев человеческий
Атлантида: Затерянный мир

Пользователь, похожий на данного:

        user_id  age  income  sex  kids_flg
739761   177374    2       5    0         1

Он смотрел следующие фильмы:

Дикая река
Подслушано
Волшебное зеркало, или Двойные неприятности
Спасибо, папа
Из Африки
Я, снова я и Ирэн
Без границ
Ищейка
Единичка


In [306]:
recs

['Содержанки',
 'Клиника счастья',
 'Мирный воин',
 'Дуров',
 'Шугалей 2',
 'Вампиры средней полосы',
 'Медиатор']

Дуров, Гнев Человеческий и Афера - подходящие рекомендации. Не уверен на счет остальных фильмов, потому что не слышал о них, но в целом рекомендации неплохие.

Можно сделать вывод, что рекомендации далеки от идеальных и, пожалуй, у пользователя очень мало параметров, по которым ищется их схожесть, но тем не менее в какой-то мере эти рекомендации можно считать отчасти подходящими.

# Холодные пользователи

Холодным пользователям можно выдавать рекомендации на самые популярные фильмы. Обновим функцию (без print'ов) и добавим рекомендации для холодных пользователей.

In [307]:
def get_recomendations_ANN(user_id, n_nearest = 5, cold_user = False):
  recs = pd.DataFrame()
  recs_titles = []
  i = 0

  # проверяем холодный ли пользователь:
  if users_added_interactions.loc[
            users_added_interactions['user_id'] == user_id
  ].empty:
    cold_user = True


  while (len(recs) < 7):
    nearest_list = ANN_tree.get_nns_by_item(user_id, n_nearest)
    user_index = 0

    if cold_user:
      recs = users_added_interactions.groupby(
          ['item_id']).size().sort_values(ascending=False)
      recs = pd.DataFrame({'item_id':recs.index, 'watched_count':recs.values})
      recs = recs.head(7)
      break
    else:
      for id in nearest_list:
        # Чем ближе пользователь в ANN к исходному, тем рекомендации приоритетнее
        user_index+=1

        df_nearest_user_interactions = users_added_interactions.loc[
            users_added_interactions['user_id'] == id
            ]['item_id']

        for item_id in df_nearest_user_interactions:
          current_user_rec = items.loc[items['item_id'] == item_id]['title'].to_list()[0]
          df = pd.DataFrame([
            [item_id,
            users_added_interactions.loc[
                (users_added_interactions['item_id']==item_id) & (users_added_interactions['user_id']==id)
                ]['last_watch_dt'].to_list()[0],
            users_added_interactions.loc[
                (users_added_interactions['item_id']==item_id) & (users_added_interactions['user_id']==id)
                ]['weight'].to_list()[0],
            user_index
            ]],
            columns=['item_id','last_watch_dt','weight','user_index']
          )
          recs = pd.concat([recs, df])
          print(current_user_rec)

    # убираем из рекомендаций фильмы, которые пользователи не смотрели достаточно времени
    recs = recs[recs.weight >= 2] 

    # сортируем по времени
    recs = recs.sort_values(['last_watch_dt', 'user_index'], ascending=[False, False])
    
    # повторяющиеся фильмы удаляем
    recs = recs.drop_duplicates(subset=['item_id']) 

    recs = recs.head(7)

    i+=1
    if i > 10:
      print('TIMEOUT: Не удалось найти полный список рекомендаций')

  recs = recs['item_id'].to_list()
  for item_id in recs:
    recs_titles.append(items.loc[items['item_id'] == item_id]['title'].to_list()[0])
  
  return recs_titles

Получен топ популярных рекомендаций:

In [308]:
get_recomendations_ANN(1097559, 5, True)

['Хрустальный',
 'Клиника счастья',
 'Гнев человеческий',
 'Девятаев',
 'Секреты семейной жизни',
 'Прабабушка легкого поведения',
 'Подслушано']